In [1]:
filename = 'FACTURACIÓN SEM 10 (01 A 06 DE MARZO) 20220307.xlsx'

# Libraries

In [2]:
import pandas as pd
import pyodbc
import os
from datetime import datetime

pd.io.formats.excel.ExcelFormatter.header_style = None

# Parameters

In [3]:
path = r'\\NASPRO.infovisiontv.com\DGI\DGI Banco Central\00 CIERRES SEMANALES\{0} {1}'
year = datetime.today().year
week = datetime.today().isocalendar()[1]
path = path.format(year, str(week).zfill(2))

# Import Data

In [4]:
%%time
df = pd.read_excel(path + '\\' + filename, sheet_name='YOSAF')

Wall time: 11.5 s


In [5]:
df.shape

(7777, 47)

In [6]:
df.tail()

,Doc.fact.,Posición,Documento de ventas,Posición.1,Referencia cliente,Material,Cd.Barra,Denomin.posición,Solicitante,Nombre,...,Nombre empresa,Organización ventas,Centro,Almacén,Lote,FeCaduc/FePreferCons,Motivo pedido,Descripción Motivo Pedido,Valor neto.1,Moneda del documento
7772,196260036,141,15310350,140,50,8100004723,6.502400e+11,SUEROX ADUL 8IONES COCO 630 ML 1020,1100813,ABARROTES SAN JUAN HEHE SA DE CV,...,GLI GENOMMA LAB INT,VNMX,MX01,MX16,2059023111,2023-08-30,NaN,NaN,1556.40,MXN
7773,196260036,142,15310350,140,50,8100004723,6.502400e+11,SUEROX ADUL 8IONES COCO 630 ML 1020,1100813,ABARROTES SAN JUAN HEHE SA DE CV,...,GLI GENOMMA LAB INT,VNMX,MX01,MX16,2036023146,2023-08-30,NaN,NaN,45291.24,MXN
7774,196260036,143,15310350,140,50,8100004723,6.502400e+11,SUEROX ADUL 8IONES COCO 630 ML 1020,1100813,ABARROTES SAN JUAN HEHE SA DE CV,...,GLI GENOMMA LAB INT,VNMX,MX01,MX16,2036023145,2023-08-30,NaN,NaN,4202.28,MXN
7775,196260036,150,15310350,150,50,8100004861,6.502401e+11,SUEROX ADUL 8IONES SABILA 630ML DIC20/12,1100813,ABARROTES SAN JUAN HEHE SA DE CV,...,GLI GENOMMA LAB INT,VNMX,MX01,MX16,2032022202,2023-07-26,NaN,NaN,39376.92,MXN
7776,196260036,151,15310350,150,50,8100004861,6.502401e+11,SUEROX ADUL 8IONES SABILA 630ML DIC20/12,1100813,ABARROTES SAN JUAN HEHE SA DE CV,...,GLI GENOMMA LAB INT,VNMX,MX01,MX16,2034022204,2023-07-26,NaN,NaN,15097.08,MXN


# Data Cleaning

df = df[:df.shape[0]-6]

In [7]:
df.isnull().sum()

Doc.fact.                       0
Posición                        0
Documento de ventas             0
Posición.1                      0
Referencia cliente              0
Material                        0
Cd.Barra                      712
Denomin.posición                0
Solicitante                     0
Nombre                          0
RFC                          2231
Nº cuenta anterior            754
Grupo de cuentas                0
Grupo de tesorería              0
Destinatario mcía.              0
Nombre.1                        0
Cantidad facturada              0
Un.medida venta                 0
Ctd.Fct.UMB                     0
Un.Med.Base                     0
Prc.Unitario                    0
Imp.Base                        0
Imp.Descuento                   0
Valor neto                      0
Imp.Impuesto                    0
Imp.IEPS                        0
Imp.Total                       0
Centro de beneficio             0
Descripción CeBe                0
Tp.Unidades   

In [8]:
new_cols = [
     'Factura',
     'Posicion',
     'DocumentoVentas',
     'PosicionDV',
     'NumeroPedCliente',
     'Material',
     'CodigoBarras',
     'Denominacion',
     'Solicitante',
     'Nombre',
     'RFC',
     'CNAC',
     'GrupoCuentas',
     'GrupoTesoreria',
     'DestinatarioMcia',
     'NombreDestinatario',
     'CantidadFacturada',
     'UnidadMedida',
     'CtdFactUMB',
     'UnidadMedidaBase',
     'PrecioUnitario',
     'ImpuestoBase',
     'ImpuestoDescuento',
     'ValorNeto',
     'ImporteImpuesto',
     'ImpuestoIEPS',
     'ImpuestoTotal',
     'CentroBeneficio',
     'DescripcionCebe',
     'TipoUnidades',
     'FechaFactura',
     'GrupoArticulos',
     'DenominacionArticulos',
     'ClaseFactura',
     'DenominacionTipoDocto',
     'CambioContable',
     'Sociedad',
     'Nombreempresa',
     'OrganizacionVentas',
     'Centro',
     'Almacen',
     'Lote',
     'FechaCaducidad',
     'MotivoPedido',
     'DescMotivoPedido',
     'ValorNetoAbs',
     'Moneda'
]

In [9]:
df.columns = new_cols

In [10]:
cols_int = [
    'Factura',
    'Posicion',
    'DocumentoVentas',
    'PosicionDV',
    'Material',
    'Solicitante',
    'CtdFactUMB',
    'ImpuestoDescuento',
    'ImpuestoIEPS',
    'GrupoArticulos'
]

In [11]:
for col in cols_int:
    df[col] = df[col].map(int)

In [12]:
df[['FechaCaducidad', 'FechaCaducidad']].dtypes

FechaCaducidad    datetime64[ns]
FechaCaducidad    datetime64[ns]
dtype: object

# Analysis

In [13]:
df[['FechaCaducidad', 'FechaCaducidad']].min()

FechaCaducidad   2022-06-30
FechaCaducidad   2022-06-30
dtype: datetime64[ns]

In [14]:
df[['FechaCaducidad', 'FechaCaducidad']].max()

FechaCaducidad   2027-02-28
FechaCaducidad   2027-02-28
dtype: datetime64[ns]

# Export Data

In [15]:
df.to_excel(path + '\\Cargar ' + filename.split('.')[0] + '.xlsx',
            index=False,
            sheet_name='carga')

In [16]:
df.shape

(7777, 47)

# Homologar productos:

In [17]:
material_ean = [
    (8100004127, 650240052580), 
    (8100005449, 650240055870)
]

In [18]:
material_ean = {
    'Material':[8100004127, 8100005449], 
    'EAN':[650240052580, 650240055870]
}

In [19]:
df[['Material', 'Nombre', 'Denominacion']][df['Material'].isin(material_ean['Material'])].drop_duplicates()

,Material,Nombre,Denominacion
166,8100005449,CASA MARZAM SA DE CV,GOICO DOBLE MENTOL GLB 0721 400ML/12
1165,8100005449,WAL MART SUPERCENTER SUPERCENTER,GOICO DOBLE MENTOL GLB 0721 400ML/12
2197,8100005449,FUXS DISTRIBUCIONES SA DE CV,GOICO DOBLE MENTOL GLB 0721 400ML/12
2427,8100005449,SUPERMERCADOS INTERNACIONALES HEB S,GOICO DOBLE MENTOL GLB 0721 400ML/12
3458,8100005449,TIENDAS CHEDRAUI S.A. DE C.V.,GOICO DOBLE MENTOL GLB 0721 400ML/12
5072,8100005449,GRUPO COMERCIAL DSW SA DE CV,GOICO DOBLE MENTOL GLB 0721 400ML/12
5248,8100005449,RAMA FARMACEUTICA SA DE CV,GOICO DOBLE MENTOL GLB 0721 400ML/12
6076,8100005449,FARMACIA GUADALAJARA S.A. DE C.V.,GOICO DOBLE MENTOL GLB 0721 400ML/12
6366,8100005449,TIENDAS SORIANA S.A. DE C.V.,GOICO DOBLE MENTOL GLB 0721 400ML/12


In [20]:
df.columns

Index(['Factura', 'Posicion', 'DocumentoVentas', 'PosicionDV',
       'NumeroPedCliente', 'Material', 'CodigoBarras', 'Denominacion',
       'Solicitante', 'Nombre', 'RFC', 'CNAC', 'GrupoCuentas',
       'GrupoTesoreria', 'DestinatarioMcia', 'NombreDestinatario',
       'CantidadFacturada', 'UnidadMedida', 'CtdFactUMB', 'UnidadMedidaBase',
       'PrecioUnitario', 'ImpuestoBase', 'ImpuestoDescuento', 'ValorNeto',
       'ImporteImpuesto', 'ImpuestoIEPS', 'ImpuestoTotal', 'CentroBeneficio',
       'DescripcionCebe', 'TipoUnidades', 'FechaFactura', 'GrupoArticulos',
       'DenominacionArticulos', 'ClaseFactura', 'DenominacionTipoDocto',
       'CambioContable', 'Sociedad', 'Nombreempresa', 'OrganizacionVentas',
       'Centro', 'Almacen', 'Lote', 'FechaCaducidad', 'MotivoPedido',
       'DescMotivoPedido', 'ValorNetoAbs', 'Moneda'],
      dtype='object')

In [21]:
df[['Material', 'Nombre', 'Denominacion', 'PrecioUnitario']][df['Material'].isin(material_ean['Material'])].drop_duplicates()

,Material,Nombre,Denominacion,PrecioUnitario
166,8100005449,CASA MARZAM SA DE CV,GOICO DOBLE MENTOL GLB 0721 400ML/12,65.94
1165,8100005449,WAL MART SUPERCENTER SUPERCENTER,GOICO DOBLE MENTOL GLB 0721 400ML/12,59.69
2197,8100005449,FUXS DISTRIBUCIONES SA DE CV,GOICO DOBLE MENTOL GLB 0721 400ML/12,64.40
2427,8100005449,SUPERMERCADOS INTERNACIONALES HEB S,GOICO DOBLE MENTOL GLB 0721 400ML/12,70.59
3458,8100005449,TIENDAS CHEDRAUI S.A. DE C.V.,GOICO DOBLE MENTOL GLB 0721 400ML/12,70.59
5072,8100005449,GRUPO COMERCIAL DSW SA DE CV,GOICO DOBLE MENTOL GLB 0721 400ML/12,65.18
5248,8100005449,RAMA FARMACEUTICA SA DE CV,GOICO DOBLE MENTOL GLB 0721 400ML/12,73.31
6076,8100005449,FARMACIA GUADALAJARA S.A. DE C.V.,GOICO DOBLE MENTOL GLB 0721 400ML/12,69.51
6366,8100005449,TIENDAS SORIANA S.A. DE C.V.,GOICO DOBLE MENTOL GLB 0721 400ML/12,70.59
6638,8100005449,WAL MART SUPERCENTER SUPERCENTER,GOICO DOBLE MENTOL GLB 0721 400ML/12,53.70


In [22]:
conn_appsgl = pyodbc.connect('Driver={SQL Server};'
                      'Server=APPSGL;'
                      'Database=Gnm_CatMaestros;'
                      'Trusted_Connection=yes;')

conn_appsgl = pyodbc.connect('Driver={SQL Server};'
                      'Server=APPSGL;'
                      'Database=Gnm_CatMaestros;'
                      'Trusted_Connection=yes;')

SELECT Material FROM [APPSGL].[Gnm_CatMaestros].[dbo].[GnmHomologaMaterialSAP]

SyntaxError: invalid syntax (Temp/ipykernel_12872/2470223518.py, line 11)

In [ ]:
query = '''INSERT INTO GnmHomologaMaterialSAP
           ([Material]
           ,[ProPstID]
           ,[Denominacion]
           ,[CodigoBarras]
           ,[FechaAudit])
     VALUES
           (8100004127
           ,{1}
           ,{2}
           ,650240052580
           ,{4})
        '''

# Homologar clientes